In [10]:
#----------------------------------------------------------------
#---Patrick Fiske------------------------------------------------
#---Chirp SFCW Ascan Processing Code-----------------------------
#---7/22/2020----------------------------------------------------
#----------------------------------------------------------------
#clear all
#close all
## load data and parameters
import numpy as np 

#C1 = rawdata; # enter raw data here


C1 = np.random.random_sample(1000)


In [12]:
start_freq  = 800e6; # enter the start frequency
end_freq    = 4e9; # enter end frequency
sample_rate = 16e6; # enter the sampling rate of the radar

fc = start_freq; # initial frequency
BWf = 8.4e9; # frequency bandwidth
N = len(C1[1:]); # this is the length of your rx buffer
fs = sample_rate; # sub-pulse sample rate
fs2 = 16.8e9; # sample rate to scale wideband signal
To = N/fs; # sub-pulse period
N2=(75346063/2); # half the length of rx buffer
To2 = N2/fs2; # period for scaling
c = (3e8); # speed of light


t = np.linspace(-To/2,To/2,To/(N-1)) #time vector
fmax = 4.2e9; #wideband bandwidth *2?



ttmeter = np.linspace(0,c*To2,c*To2/(N2-1)) #time domain vector - meters

ttinch = np.linspace(0,39.3701*c*To2,39.3701*c*To2/(N2-1)); #time domain vector - inches

f =np.linspace(-fmax,fmax,BWf/(2*N2-1)); #frequency domain - centered at zero

In [15]:
import pycuda.gpuarray as gpuarray


import numpy as np
# Not to be confused with functions to be used on the Windows OS
# These window functions are similar to those found in the Windows toolbox of MATLAB
# Note that numpy has a couple of Window functions already:
# See: hamming, bartlett, blackman, hanning, kaiser
 
def tukeywin(window_length, alpha=0.5):
    '''The Tukey window, also known as the tapered cosine window, can be regarded as a cosine lobe of width \alpha * N / 2
    that is convolved with a rectangle window of width (1 - \alpha / 2). At \alpha = 1 it becomes rectangular, and
    at \alpha = 0 it becomes a Hann window.
 
    We use the same reference as MATLAB to provide the same results in case users compare a MATLAB output to this function
    output
 
    Reference
    ---------
    http://www.mathworks.com/access/helpdesk/help/toolbox/signal/tukeywin.html
 
    '''
    # Special cases
    if alpha <= 0:
        return np.ones(window_length) #rectangular window
    elif alpha >= 1:
        return np.hanning(window_length)
 
    # Normal case
    x = np.linspace(0, 1, window_length)
    w = np.ones(x.shape)
 
    # first condition 0 <= x < alpha/2
    first_condition = x<alpha/2
    w[first_condition] = 0.5 * (1 + np.cos(2*np.pi/alpha * (x[first_condition] - alpha/2) ))
 
    # second condition already taken care of
 
    # third condition 1 - alpha / 2 <= x <= 1
    third_condition = x>=(1 - alpha/2)
    w[third_condition] = 0.5 * (1 + np.cos(2*np.pi/alpha * (x[third_condition] - 1 + alpha/2))) 
 
    return w



w = gpuArray(tukeywin(277574)); #turkey window length of rx buffer



ModuleNotFoundError: No module named 'pycuda'

In [ ]:
fs_up = 15e6; #used to calculate upsampling, L
Tp = 40001 / fs; #period for upsampling -
L = (fs)*Tp; #upsampling factor
Lup = 2 * L;
n_zeros = np.around(Lup / 2);
Dn1 = zeros(1,75346063,'gpuArray'); # pre-allocation of wideband vector

## Frequency Stacking
# set up parameters for zero padding
#tic
delta_f = 0;

#this first loop loads the data
for n = 1:201
    rx(n,:) = gpuArray(C1(n,2,:)); # allocate to gpu for faster processing
    rx(n,:) = squeeze(rx(n,:)); # remove extra dimension
    rx(n,:) = rx(n,:)-mean(rx(n,:)); # remove dc component
    b(n) = max(abs(rx(n,:))); # variable for gain tables
    rx2(n,:) = gpuArray(C1(n,1,:));
    rx2(n,:) = squeeze(rx2(n,:));
    rx2(n,:) = rx2(n,:)-mean(rx2(n,:));
    a(n) = max(abs(rx2(n,:)));
end

#this second loop performs frequency stacking
for n in 1:201
    DFT_rx = gpuArray((fft(rx(n,:)))./363); #fft on gpu
    DFT_rx = DFT_rx(1000:278573);
    DFT_reference = gpuArray((fft(rx2(n,:)))./363); # repeat for reference
    DFT_reference = DFT_reference(1000:278573);
    DFT_D0 = (DFT_rx .* conj(DFT_reference));
    DFT_D0_shifted = gpuArray(fftshift(DFT_D0)); # fftshift the result
    DFT_D0_shifted = [zeros(1, n_zeros) DFT_D0_shifted zeros(1, 75028488)]; # pad
    DFT_D0_shifted = circshift(DFT_D0_shifted, 23746744 + round(delta_f)); # shift
    delta_f = delta_f+2*69000;

    count2 =n # displays counter
    Dn1 = Dn1+DFT_D0_shifted; #sum each shifted pulse


#toc


In [ ]:
## Plotting Frequency Response and A Scans
D1 = gather(Dn1); # bring data back to processor
D = D1./max(D1); # normalized data
L = length(D); # length for use in filter

h = hamming(L)'; # hamming filter

set(0,'defaultfigurecolor',[1 1 1])
figure()
plot(f,(20*log10(abs(h.*D.*GLS)))) # hamming * data * gls filter
title('Reconstructed SWW');xlabel('Frequency(Hz)');ylabel('Amplitude(dB)');
ylim([-100 0])
grid on
hold on

#creat time domain signal
y=(fftshift(ifft(h.*D.*GLS_yan))); # ifft of the filtered data
y = y/max(y); # normalize data
y = circshift(y,-77);
y = y(:,[37672972:75346002]); # matched filter output

figure()
plot(ttmeter,((abs(y)))); title('A Scan');
xlim([0 10])
ylim([0 1.1])
grid on

figure()
plot(ttinch,(abs(y))); title('A Scan'); xlabel('Distance(Inches)'); ylabel('Amplitude')
xlim([0 100])
ylim([0 1])
grid on
